# Fase 03: Optimización del modelo en HAR (Hailo Archive)

In [1]:
# Imports

import json
import os
import gc

import numpy as np
import tensorflow as tf
from IPython.display import SVG
from matplotlib import patches
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.python.eager.context import eager_mode
import torch
import onnx
import onnxruntime as ort
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from itertools import islice

# Funciones y parámetros de la CNN base
import sys
sys.path.append('../src/models/')
from cnn21_pix import *

# import the hailo sdk client relevant classes
from hailo_sdk_client import ClientRunner, InferenceContext

2025-05-20 13:10:00.749086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-20 13:10:01.438100: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/pablo/anaconda3/envs/hailo_gpu_env/lib/python3.8/site-packages/nvidia/dali/backend.py:77: Warning: DALI 1.49 is the last release to support Python 3.8 Please update your environment to use Python 3.9, or newer.
  deprecation_warning(


## 1. Optimización detallada

### 1.1. Carga de datos

In [2]:
# Definir parámetros

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'

# Queremos usar todos los datos para la inferencia
SAMPLES=[0,0]
PAD=1
AUM=0

In [3]:
# Carga de datos

(datos,H,V,B)=read_raw(DATASET)
(truth,H1,V1)=read_pgm(GT)

# Durante la ejecucion de la red vamos a coger patches de tamano cuadrado
sizex=32; sizey=32 

# Hacemos padding en el dataset para poder aprovechar hasta el borde
if(PAD):
    datos=torch.FloatTensor(np.pad(datos,((sizey//2,sizey//2),(sizex//2,sizex//2),(0,0)),'symmetric'))
    H=H+2*(sizex//2); V=V+2*(sizey//2)
    truth=np.reshape(truth,(-1,H1))
    truth=np.pad(truth,((sizey//2,sizey//2),(sizex//2,sizex//2)),'constant')
    H1=H1+2*(sizex//2); V1=V1+2*(sizey//2)
    truth=np.reshape(truth,(H1*V1))
    
# Necesitamos los datos en band-vector para hacer convoluciones
datos=np.transpose(datos,(2,0,1))

# Seleccionar conjunto de test (en este caso es una predicción)
(train,val,test,nclases,nclases_no_vacias)=select_training_samples(truth,H,V,sizex,sizey,SAMPLES)
dataset_test=HyperDataset(datos,truth,test,H,V,sizex,sizey)
print('  - test dataset:',len(dataset_test))

# Dataloader
batch_size=100 # defecto 100
# Indicamos shuffle = True porque no se evalúa todo el dataset
test_loader=DataLoader(dataset_test,batch_size,shuffle=True)

* Read dataset: ../data/imagenes_rios/oitaven_river.raw
  B: 5 H: 6722 V: 6689
  Read: 224817290
* Read GT: ../data/imagenes_rios/oitaven_river.pgm
  H: 6722 V: 6689 depth: 10
  Read: 44963458
* Select training samples
  nclasses: 10
  Class  # :   total | train |   val |    test
  Class  1 :  309248 |     0 |     0 |  309248
  Class  2 :  113324 |     0 |     0 |  113324
  Class  3 :   79152 |     0 |     0 |   79152
  Class  4 :   43861 |     0 |     0 |   43861
  Class  5 :  128022 |     0 |     0 |  128022
  Class  6 :   78785 |     0 |     0 |   78785
  Class  7 : 2428482 |     0 |     0 | 2428482
  Class  8 : 1829360 |     0 |     0 | 1829360
  Class  9 :  193884 |     0 |     0 |  193884
  Class 10 :  863061 |     0 |     0 |  863061
  - test dataset: 6067179


In [4]:
# Cargar el modelo convertido a ONNX

ort_session = ort.InferenceSession("../results/model_cnn21.onnx")

# Cargar el modelo en HAR

model_name = "../results/model_cnn21"
hailo_model_har_name = f"{model_name}_hailo_model.har"
assert os.path.isfile(hailo_model_har_name), "Please provide valid path for HAR file"
runner = ClientRunner(har=hailo_model_har_name, hw_arch="hailo8l")

### 1.2 Evaluar el modelo en har sin optimización

In [9]:
# Comprobacion rapida de la diferencia de precisión

# Crear tensor de entrada de prueba
input_tensor = torch.randn(1, B, sizex, sizey).to("cpu")
input_np = input_tensor.cpu().numpy()

# Realizar la inferencia en ONNX
output_onnx = ort_session.run(None, {'input': input_np})[0]

# print(output_onnx)

# Realizar la inferencia en HAR
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    input_har = np.transpose(input_np, (0, 2, 3, 1))

    # Realizar la inferencia en el modelo .har
    output_har = runner.infer(ctx, input_har)

    # print(output_har) 

# Comparar las diferencias
error = np.abs(output_onnx - output_har).mean()
print(f'Error medio entre PyTorch y ONNX: {error}')

2025-05-19 13:48:41.842469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-19 13:48:41.848134: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-19 13:48:41.848337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Using 1 GPU for inference


2025-05-19 13:48:44.045210: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-19 13:48:44.344213: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-05-19 13:48:48.440281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2025-05-19 13:48:49.472938: W tensorflow/core/kernels/gpu_utils.cc:70] Failed to check cudnn convolutions for out-of-bounds reads and writes with an error message: 'Failed to load in-memory CUBIN: CUDA_ERROR_INVALID_IMAGE: device kernel image is invalid'; skipping this check. This only means that we won't check cudnn for out-of-boun

Error medio entre PyTorch y ONNX: 4.57763690064894e-06


In [6]:
'''
# El bucle de inferencia tiene un memory leak que proviene del SDK de hailo con el emulador NATIVE
# Se realiza la evaluacion, por tanto, con un unico batch

# Inferencia

output=np.zeros(H*V,dtype=np.uint8)

# Medir el tiempo de ejecución
start_time = time.time()

# Realizar la predicción
with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
    total=0
    for (inputs, labels) in test_loader:
        # Convertir inputs a un formato adecuado para Hailo (numpy array)
        # Cambiamos de (batch_size, B, sizex, sizey) a (batch_size, sizey, sizex, B)
        inputs_np = np.transpose(inputs.numpy(), (0, 3, 2, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_np)
        
        predicted=np.argmax(native_res, axis=-1)
        predicted=predicted.squeeze()
        
        # Asignar las predicciones al array de salida
        for i in range(len(predicted)):
            output[test[total+i]]=np.uint8(predicted[i]+1)
        total+=labels.size(0)

        # Mostrar el progreso
        if(total%100000==0): print('  Test:',total,'/',len(dataset_test))
        
end_time = time.time()

print("Prediction time: {:.4f} seconds".format(end_time - start_time))

'''

'\n# El bucle de inferencia tiene un memory leak que proviene del SDK de hailo con el emulador NATIVE\n# Se realiza la evaluacion, por tanto, con un unico batch\n\n# Inferencia\n\noutput=np.zeros(H*V,dtype=np.uint8)\n\n# Medir el tiempo de ejecución\nstart_time = time.time()\n\n# Realizar la predicción\nwith runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:\n    total=0\n    for (inputs, labels) in test_loader:\n        # Convertir inputs a un formato adecuado para Hailo (numpy array)\n        # Cambiamos de (batch_size, B, sizex, sizey) a (batch_size, sizey, sizex, B)\n        inputs_np = np.transpose(inputs.numpy(), (0, 3, 2, 1))\n\n        # Realizar la inferencia en el modelo .har\n        native_res = runner.infer(ctx, inputs_np)\n        \n        predicted=np.argmax(native_res, axis=-1)\n        predicted=predicted.squeeze()\n        \n        # Asignar las predicciones al array de salida\n        for i in range(len(predicted)):\n            output[test[total+i]]=np.uint

In [7]:
# Evaluar el modelo en HAR sin optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_NATIVE) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:18.366492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1540.91entries/s]


[Iteración 1] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:18.861796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2050.92entries/s]


[Iteración 2] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:17:19.306366: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2172.28entries/s]


...

[Iteración 98] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:02.892880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2248.20entries/s]


[Iteración 99] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:03.326304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2316.19entries/s]

[Iteración 100] Error medio: 0.0000 ; Coincidencias: 100/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.000000
Precisión global: 100.00%


### 1.3 Aplicar modificaciones de optimización al modelo y evaluarlo

In [10]:
# Crear un model script para el proceso de optimización

model_script_lines = [
    # Add normalization layer with mean [123.675, 116.28, 103.53] and std [58.395, 57.12, 57.375])
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # For multiple input nodes:
    # {normalization_layer_name_1} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_1_from_hn})\n',
    # {normalization_layer_name_2} = normalization([list of means per channel], [list of stds per channel], {input_layer_name_2_from_hn})\n',
    # ...
]

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script("".join(model_script_lines))
runner.optimize_full_precision()

[warning] Model script is empty
[info] Loading model script commands to model_cnn21 from string
[warning] Model script is empty
[warning] DEPRECATION WARNING: Optimizing in full precision will require calibration data in the near future, to allow more accurate optimization algorithms which require inference on actual data.


In [9]:
# Evaluar el modelo en HAR con optimizacion

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_FP_OPTIMIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:34.587670: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 270.39entries/s]


[Iteración 1] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:35.459731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2039.36entries/s]


[Iteración 2] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:18:35.912833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2168.35entries/s]


...

[Iteración 98] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:19:19.253537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2241.04entries/s]


[Iteración 99] Error medio: 0.0000 ; Coincidencias: 100/100


Inference: 0entries [00:00, ?entries/s]2025-04-18 03:19:19.710452: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2302.29entries/s]

[Iteración 100] Error medio: 0.0000 ; Coincidencias: 100/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.000000
Precisión global: 100.00%


### 1.4 Cuantizar el modelo y evaluarlo

In [5]:
# Crear un dataset de calibración
# The original images are being used, just as the input to the SDK_FP_OPTIMIZED emulator
total_images = 1050

calib_dataset = np.zeros((total_images, sizex, sizey, B), dtype = np.float32)

count = 0

# Seleccionamos batches del dataloader para guardarlos en el dataset
for inputs, _ in test_loader:
    for img in inputs:
        if count >= total_images:
            break
            
        # Los inputs son de la forma (batch_size, B, sizex, sizey)
        img_np = img.numpy()
        # Trasponemos los inputs a formato (batch_size, sizex, sizey, B)
        img_har = np.transpose(img_np, (1, 2, 0))

        calib_dataset[count] = img_har
        count += 1
        
    if count >= total_images:
        break

In [6]:
# Cuantizar el modelo con el dataset de calibración

# For calling Optimize, use the short version: runner.optimize(calib_dataset)
# A more general approach is being used here that works also with multiple input nodes.
# The calibration dataset could also be a dictionary with the format:
# {input_layer_name_1_from_hn: layer_1_calib_dataset, input_layer_name_2_from_hn: layer_2_calib_dataset}
hn_layers = runner.get_hn_dict()["layers"]
print("Input layers are: ")
print([layer for layer in hn_layers if hn_layers[layer]["type"] == "input_layer"])  # See available input layer names
calib_dataset_dict = {"model_cnn21/input_layer1": calib_dataset}  # In our case there is only one input layer

optimization_level = 1
alls_lines = [
    # "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n",
    # Batch size is 8 by default; 2 was used for stability on PCs with low amount of RAM / VRAM
    f"model_optimization_flavor(optimization_level={optimization_level}, compression_level=0, batch_size=8)\n",
    # The following line is needed because resnet_v1_18 is a really small model, and the compression_level is always reverted back to 0.'
    # To force using compression_level with small models, the following line should be used (compression level=4 equals to 80% 4-bit):
    # "model_optimization_config(compression_params, auto_4bit_weights_ratio=0.2)\n",
    # The application of the compression could be seen by the [info] messages: "Assigning 4bit weight to layer .."
]

runner.load_model_script("".join(alls_lines))

runner.optimize(calib_dataset_dict)

Input layers are: 
['model_cnn21/input_layer1']
[info] Loading model script commands to model_cnn21 from string
[info] Starting Model Optimization


2025-05-20 13:10:35.814797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:10:35.816434: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:10:35.816654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Model received quantization params from the hn


2025-05-20 13:10:39.566949: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:10:39.568780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:10:39.568999: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] MatmulDecompose skipped
[info] Starting Mixed Precision
[info] Model Optimization Algorithm Mixed Precision is done (completion time is 00:00:00.02)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration


...

[info] Model Optimization Algorithm Bias Correction is done (completion time is 00:00:07.24)
[info] Adaround skipped
[info] Quantization-Aware Fine-Tuning skipped


2025-05-20 13:11:02.097757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:11:02.099570: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:11:02.099777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Starting Layer Noise Analysis


Full Quant Analysis:   0%|                                | 0/2 [00:00<?, ?iterations/s]2025-05-20 13:11:06.918501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-05-20 13:11:06.938374: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [8,32,32,5]
	 [[{{node Placeholder/_0}}]]
2025-05-20 13:11:14.926440: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
Full Quant Analysis: 100%|████████████████████████| 2/2 [00:08<00:00,  4.39s/iterations]
2025-05-20 13:11:15.558716: W t

[info] Model Optimization Algorithm Layer Noise Analysis is done (completion time is 00:00:09.12)
[info] Output layers signal-to-noise ratio (SNR): measures the quantization noise (higher is better)
[info] 	model_cnn21/output_layer1 SNR:	34.29 dB
[info] Model Optimization is done


In [7]:
# Evaluar el modelo en HAR cuantizado

total_error = 0
total_matches = 0
total_elements = 0

num_iters = 100

for i in range(num_iters):

    inputs, labels = next(iter(test_loader))
    
    inputs_np = inputs.numpy()

    # Inferencia para el modelo en ONNX
    outputs = ort_session.run(None, {'input': inputs_np})
    preds_onnx=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones

    # Inferencia para el modelo en HAR
    with runner.infer_context(InferenceContext.SDK_QUANTIZED) as ctx:
        inputs_har = np.transpose(inputs.numpy(), (0, 2, 3, 1))

        # Realizar la inferencia en el modelo .har
        native_res = runner.infer(ctx, inputs_har)
        
        preds_har=np.argmax(native_res, axis=-1).squeeze()

    # Comparar la diferencia entre ONNX y har utilizando el error medio absoluto
    error = np.abs(preds_onnx - preds_har).mean()
    total_error += error
    
    # Comparacion exacta
    matches = np.sum(preds_onnx == preds_har)
    total_matches += matches
    total_elements += preds_onnx.size
    
    print(f"[Iteración {i+1}] Error medio: {error:.4f} ; Coincidencias: {matches}/{preds_onnx.size}")
    
# Resultados globales
mean_error = total_error / num_iters
accuracy = total_matches / total_elements

print(f"Resultados globales para {total_elements} predicciones ({num_iters} iteraciones):")
print(f"Error medio absoluto (promedio): {mean_error:.6f}")
print(f"Precisión global: {accuracy*100:.2f}%")

2025-05-20 13:08:08.221115: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:08:08.222441: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-20 13:08:08.222654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[info] Using 1 GPU for inference


2025-05-20 13:08:09.882796: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-20 13:08:10.970772: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-05-20 13:08:15.217457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2025-05-20 13:08:15.734470: W tensorflow/core/kernels/gpu_utils.cc:70] Failed to check cudnn convolutions for out-of-bounds reads and writes with an error message: 'Failed to load in-memory CUBIN: CUDA_ERROR_INVALID_IMAGE: device kernel image is invalid'; skipping this check. This only means that we won't check cudnn for out-of-boun

[Iteración 1] Error medio: 0.0000 ; Coincidencias: 100/100
[info] Using 1 GPU for inference


2025-05-20 13:08:16.230155: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-20 13:08:16.312910: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1974.64entries/s]


...

[Iteración 98] Error medio: 0.0500 ; Coincidencias: 98/100
[info] Using 1 GPU for inference


2025-05-20 13:09:12.344050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-20 13:09:12.434187: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 2037.72entries/s]


[Iteración 99] Error medio: 0.0300 ; Coincidencias: 98/100
[info] Using 1 GPU for inference


2025-05-20 13:09:12.902356: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:1279] could not retrieve CUDA device count: CUDA_ERROR_NOT_INITIALIZED: initialization error
Inference: 0entries [00:00, ?entries/s]2025-05-20 13:09:12.989962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
Inference: 104entries [00:00, 1911.56entries/s]

[Iteración 100] Error medio: 0.0200 ; Coincidencias: 99/100
Resultados globales para 10000 predicciones (100 iteraciones):
Error medio absoluto (promedio): 0.041600
Precisión global: 98.16%


In [7]:
# Guardar el modelo cuantizado
# Let's save the runner's state to a Quantized HAR
quantized_model_har_path = f"{model_name}_quantized_model_o{optimization_level}.har"
runner.save_har(quantized_model_har_path)

[info] Saved HAR to: /home/pablo/Documentos/Enxeñaría Informática/Cuarto/2º Cuatrimestre/Traballo Fin de Grao/hyper-rpi/results/model_cnn21_quantized_model_o1.har
